In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [2]:
data = pd.read_csv("Stock_price.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-04-18,27.222500,27.237499,26.735001,26.870001,24.992023,243286000
1,2016-04-19,26.969999,27.000000,26.557501,26.727501,24.859484,129539600
2,2016-04-20,26.660000,27.022499,26.514999,26.782499,24.910643,122444000
3,2016-04-21,26.732500,26.732500,26.379999,26.492500,24.640911,126210000
4,2016-04-22,26.252501,26.620001,26.155001,26.420000,24.573479,134732400


In [3]:
df = data[["Adj Close"]].copy()

In [4]:
df["Target"] = data.Close.shift(-1)

In [5]:
df.dropna(inplace=True)

In [6]:
df.head(5)

,Adj Close,Target
0,24.992023,26.727501
1,24.859484,26.782499
2,24.910643,26.492500
3,24.640911,26.420000
4,24.573479,26.270000


In [7]:
size = int(len(df)*0.8)
train, test = df[:size], df[size:]
train.shape, test.shape

((1006, 2), (252, 2))

In [9]:
def xgb_predict(train, val):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X, y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [10]:
predictions = []

history = train.copy()

for i in range(len(test)):
    test_X, test_y = test.iloc[i, :-1], test.iloc[i, -1]
    
    pred = xgb_predict(train, test_X.iloc[ 0])
    predictions.append(pred)

    train.append(test.iloc[i])

error = mean_squared_error(test.iloc[:, -1], predictions, squared=False)

error

35.88196156470418